### Lymphography Dataset
Zwitter, M. & Soklic, M. (1988). Lymphography [Dataset]. UCI Machine Learning Repository. https://doi.org/10.24432/C54598.

In [1]:
import pandas as pd
from edm_cif import EDM_CIF

df = pd.read_csv('lymphography.csv', sep=',')
df_class = df['class']
df.drop('class', axis=1, inplace=True)
for c in df.columns:
    df[c] = df[c].fillna(df[c].mode()[0])

df.head()

,lymphatics,block of affere,bl. of lymph,bl. of lymph.1,by pass,extravasates,regeneration of,early uptake in,lym.nodes dimin,lym.nodes enlar,changes in lym.,defect in node,changes in node,changes in stru,special forms,dislocation,exclusion of no,no. of nodes in
0,4,2,1,1,1,1,1,2,1,2,2,2,4,8,1,1,2,2
1,3,2,1,1,2,2,1,2,1,3,3,2,3,4,2,2,2,2
2,3,2,2,2,2,2,2,2,1,4,3,3,4,8,3,2,2,7
3,3,1,1,1,1,2,1,2,1,3,3,4,4,4,3,1,2,6
4,3,1,1,1,1,1,1,1,1,2,2,4,3,5,1,2,2,1


In [2]:
df_class.value_counts()

class
2    81
3    61
4     4
1     2
Name: count, dtype: int64

In [6]:
alg = EDM_CIF(n_estimators=300, max_features=5, max_samples=500, m=2, contamination=0.04)
labels = alg.fit_predict(df)

df['class'] = df_class
df['labels'] = labels

result = df.groupby(['labels', 'class']).size().unstack(fill_value=0)
result

class,1,2,3,4
labels,,,,
-1,2,0,0,4
1,0,81,61,0


In [7]:
alg.predict_proba(df)

array([0.25317798, 0.1863441 , 0.49029872, 0.30677359, 0.12550945,
       0.31714745, 0.10973686, 0.25131454, 0.19158441, 0.20498789,
       0.27003216, 0.13021241, 0.23089758, 0.2666733 , 0.88823716,
       0.04055002, 0.19796253, 0.13210685, 0.1287342 , 0.17717596,
       0.1587379 , 0.29686541, 0.23820635, 0.18989101, 0.36103068,
       0.15898891, 0.18990777, 0.13224764, 0.09899458, 0.1568864 ,
       0.27787281, 0.10483718, 0.18856712, 0.24742327, 0.10664417,
       0.18546938, 0.83031119, 0.21207958, 0.11586665, 0.30519431,
       0.12909193, 0.38731737, 0.23274532, 0.29891614, 0.68200689,
       0.05139145, 0.19251059, 0.36148593, 0.24874447, 0.07703683,
       0.32777976, 0.22815786, 0.22737244, 0.26437327, 0.22547072,
       0.19313587, 0.09200958, 0.28726402, 0.54247879, 0.12562879,
       0.31546413, 0.25759184, 0.26987779, 0.27366115, 0.04824864,
       0.40103187, 0.24775748, 0.2585402 , 0.22106837, 0.23451962,
       0.07601304, 0.27015576, 0.38599819, 0.28393229, 0.26386